In [ ]:
import os
import requests
import json
import base64
from jinja2 import Environment, FileSystemLoader

def page_link(a):
    if ' ' in a:
        return a.replace(' ', '_')
    else:
        return a

# Use the GitHub API to retrieve the contents of the JSON files and the template directory
def get_contents(path):
    url = f"https://api.github.com/repos/harshulgupt/json_pvt"
    response = requests.get(url, headers={
        "Authorization": f"Token {os.environ['PAT']}"
    })
    if response.status_code == 200:
        contents = response.json()
        return {content["name"]: base64.b64decode(content["content"]).decode("utf-8") for content in contents}

json_contents = get_contents("jsons")
template_contents = get_contents("templates")

data = {}
for json_file in json_files:
    data[json_file] = json.loads(json_contents[f"{json_file}.json"])

# Load the templates from the retrieved contents
environment = Environment()
environment.loader = lambda name: (
    template_contents[f"{name}"].strip(),
    name,
    lambda: False
)
environment.globals['page_link'] = page_link
template = environment.get_template('contact.html.j2')

content = template.render(General=data["General"], contact=data["contact"])

#open in write mode
with open('Contact.html', mode='w', encoding='utf-8') as contact:
    contact.write(content)
